In [ ]:
from pyincore import IncoreClient
from pyincore import MappingSet, FragilityService
from pyincore.analyses.buildingdamage.buildingdamage import BuildingDamage
from pyincore.analyses.housingunitallocation import HousingUnitAllocation
from pyincore.analyses.populationdislocation import PopulationDislocation
from pyincore.analyses.housingrecovery.housingrecovery import HousingRecovery

In [ ]:
# Connect to IN-CORE service
client = IncoreClient()

# Building damage, Galveston, TX analysis

In [ ]:
hazard_type = "hurricane"
# Galveston deterministic Hurricane, 3 datasets - Kriging
hazard_id = "5fa5a228b6429615aeea4410"

# Galveston, TX Building inventory
bldg_inv_id = "60354b6c123b4036e6837ef7"  # 19k buildings with age_group

# Hurricane building mapping (with equation)
mapping_id = "602c381a1d85547cdc9f0675"
fragility_service = FragilityService(client)
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))

bldg_dmg = BuildingDamage(client)
bldg_dmg.load_remote_input_dataset("buildings", bldg_inv_id)
bldg_dmg.set_input_dataset("dfr3_mapping_set", mapping_set)

result_name = "Galveston_bldg_dmg_result"

bldg_dmg.set_parameter("fragility_key", "Hurricane SurgeLevel and WaveHeight Fragility ID Code")
bldg_dmg.set_parameter("result_name", result_name)
bldg_dmg.set_parameter("hazard_type", hazard_type)
bldg_dmg.set_parameter("hazard_id", hazard_id)
bldg_dmg.set_parameter("num_cpu", 4)

# Run Building damage
bldg_dmg.run_analysis()

# Retrieve result dataset
building_dmg_result = bldg_dmg.get_output_dataset("ds_result")

# Convert dataset to Pandas DataFrame
bdmg_df = building_dmg_result.get_dataframe_from_csv(low_memory=False)

# Display top 5 rows of output data
bdmg_df[["guid", "DS_0", "DS_1", "DS_2", "DS_3", "haz_expose"]].head(5)

# Housing Unit Allocation (HUA) analysis

In [ ]:
housing_unit_inv = "5fc6ab1cd2066956f49e7a03"
# Address point inventory
address_point_inv = "5fc6aadcc38a0722f563392e"

# Create housing allocation
hua = HousingUnitAllocation(client)

# Load input dataset
hua.load_remote_input_dataset("housing_unit_inventory", housing_unit_inv)
hua.load_remote_input_dataset("address_point_inventory", address_point_inv)
hua.load_remote_input_dataset("buildings", bldg_inv_id)

result_name = "Galveston_HUA"

seed = 1238
iterations = 1

# Set analysis parameters
hua.set_parameter("result_name", result_name)
hua.set_parameter("seed", seed)
hua.set_parameter("iterations", iterations)

# Run Housing unit allocation analysis
hua.run_analysis()

# Retrieve result dataset
hua_result = hua.get_output_dataset("result")

# Convert dataset to Pandas DataFrame
hua_df = hua_result.get_dataframe_from_csv(low_memory=False)

hua_df[["guid", "numprec", "incomegroup"]].head()

# Population Dislocation analysis

In [ ]:
# Block group data, IN-CORE_BGMAP_2021-01-19_GalvestonTX
bg_data = "603545f2dcda03378087e708" # prod
# Value loss parameters DS 0-3
value_loss = "60354810e379f22e16560dbd"

pop_dis = PopulationDislocation(client)

pop_dis.load_remote_input_dataset("block_group_data", bg_data)
pop_dis.load_remote_input_dataset("value_loss_param", value_loss)

pop_dis.set_input_dataset("building_dmg", building_dmg_result)
pop_dis.set_input_dataset("housing_unit_allocation", hua_result)

result_name = "galveston-pop-disl-results"
seed = 1111

pop_dis.set_parameter("result_name", result_name)
pop_dis.set_parameter("seed", seed)

pop_dis.run_analysis()

# Retrieve result dataset
pd_result = pop_dis.get_output_dataset("result")

# Convert dataset to Pandas DataFrame
pd_df = pd_result.get_dataframe_from_csv(low_memory=False)

pd_df[["guid", "dislocated"]].head()

# Household recovery analysis.

In [ ]:
# Additional inventory data (assesed damage, square meters)
bldg_sqft_id = "62193c19a42a3e546ae2d9f8"
# Census block groups data
census_bg_id = "62193b7ca42a3e546ae2d9f2"

# Census appraisal file; id of external Census json is required if not provide fips for API request
fips = "48167"

result_name = "Galveston_building_values_chained"

housing_rec = HousingRecovery(client)

housing_rec.set_input_dataset("population_dislocation", pd_result)
housing_rec.load_remote_input_dataset("building_area", bldg_sqft_id)
housing_rec.load_remote_input_dataset("census_block_groups_data", census_bg_id)
        
# Census appraisal data

housing_rec.set_parameter("fips", fips)
housing_rec.set_parameter("result_name", result_name)

housing_rec.run_analysis()

In [ ]:
# Retrieve result dataset
result = housing_recovery.get_output_dataset("result")

# Convert dataset to Pandas DataFrame
hr_df = result.get_dataframe_from_csv()

# Display top 5 rows of output data
hr_df.head()

## Plotting and Estimates

In [ ]:
# A csv file with the building values for the 6 years following the disaster event (year -1 denotes pre-impact 
# conditions and 0 being the impact year).
years = [-1, 0, 1, 2, 3, 4, 5, 6]

In [ ]:
# Rename column names to range from 0 to n for the aggregation

for year in years:
    hr_df.rename(columns={"year_{0}".format(year):"year_{0}".format(year+1),"index_{0}".format(year):"index_{0}".format(year+1)}, inplace=True)
    
# reshape dataframe from wide to long
hr_reshaped = pd.wide_to_long(hr_df,["year","index"], i="strctid", j="years")

# shift dataframe multiindex to columns and sort
hr_reshaped.reset_index(inplace=True)
hr_reshaped.sort_values(by=["strctid","years"], inplace=True)

# adjust year column to oringal -n to +n range by subtracting neg_count
hr_reshaped["years"] = hr_reshaped["years"] - 1
hr_reshaped.head(20)

In [ ]:
hr_collapse = hr_reshaped.groupby(["d_vacationct","years"]).aggregate({"year":np.sum})
hr_collapse.reset_index(inplace=True)
hr_collapse.style.format({"years": "{:>,.0f}"})
hr_collapse.head(20)

In [ ]:
plt.plot(hametal2018_dmg_collapse.years,hametal2018_dmg_collapse.year)
plt.title("Aggregate building values by year (dmg)")
plt.xlabel("Year")
plt.ylabel("Aggregate building value")
plt.grid(True)
plt.show()